<a href="https://colab.research.google.com/github/grndnl/w266_final_project/blob/main/Understanding_if_elements_are_part_of_an_assembly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting whether two sets of assembly names are from the same assembly

In [53]:
import os, re
from pathlib import Path
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from keras import layers
import random
!pip install transformers
from transformers import AutoTokenizer, TFAutoModel, BertTokenizer, TFBertModel
from sklearn.model_selection import train_test_split
from pprint import pprint

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Preprocess data

### Pre-process data for task

In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
data_path = "/content/drive/My Drive/W266/data_02.feather"
data = pd.read_feather(data_path)

In [56]:
data = data.dropna(subset=["assembly_name", "part_names"])
data = data.drop(columns=["assembly_id", 'assembly_description'])
#data = data.drop(columns=["assembly_id"]) This one is no longer necesary as we prove doesnt add much value
data.head()

,assembly_name,part_names
0,Lava Lamp 2,"[Blob3, Blob2, Blob1, Glass, Cap]"
1,Sample - Headphones,"[Pivot hinge, Headphone hinge, Telescope hinge..."
4,Coffee Mug,"[Mug, Lid]"
5,Dave's Handsome Mug,"[Lid, Seal, Vessel]"
9,Mechanical Pencil,"[Eraser, Pencil Lead, Rubber Grip, Gripper Rod..."


In [57]:
# Deduplicate
print(f"Tot: {len(data)}")
print(f"Unique: {len(data['assembly_name'].unique())}")
data = data[~data['part_names'].apply(tuple).duplicated()]
print(f"\nAfter dedup: {len(data)}")
print(f"Unique: {len(data['assembly_name'].unique())}")

Tot: 88886
Unique: 67834

After dedup: 61725
Unique: 54034


### Clean assembly names

In [58]:
def process_assembly_names(string):
    string = string.replace('.x_t', '')
    string = string.replace('.stp', '')
    string = string.replace('.step', '')
    string = string.replace('.zip', '')
    string = string.replace('.dwg', '')
    string = ' '.join(re.findall('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', string))  # splits camelCase into camel case
    string = ' '.join(re.split('(\s+|^)([A-Za-z]+)\d+(\s+|$)', string))  # removes number at the end
    string = string.lower()
    string = string.replace('_', ' ')
    string = string.replace('-', ' ')
    string = string.replace('[', ' ')
    string = string.replace(']', ' ')
    string = string.replace('(', ' ')
    string = string.replace(')', ' ')
    string = string.replace('?', ' ')
    string = string.replace('*', ' ')
    string = string.replace('copy of', ' ')
    string = string.replace('copy', ' ')
    string = string.replace('sample', '')
    string = string.replace('2', '')
    string = string.replace('tutorial', '')
    string = " ".join(string.split())

    return string


data['assembly_name_clean'] = data.apply(lambda row: process_assembly_names(row.assembly_name), axis=1)

In [59]:
print(f"After dedup: {len(data)}")
print(f"Unique: {len(data['assembly_name_clean'].unique())}")

After dedup: 61725
Unique: 49240


In [60]:

print(f"After dedup: {len(data)}")
print(f"Unique: {len(data['assembly_name_clean'].unique())}")

After dedup: 61725
Unique: 49240


### Clean part names

In [61]:
def process_part_names(part_list):
    part_names = []
    for string in part_list:
        if "MANIFOLD_SOLID_BREP" in string:
            return np.nan
        string = string.replace('.x_t', '')
        string = string.replace('.stp', '')
        string = string.replace('.step', '')
        string = string.replace('.dwg', '')
        string = string.replace('.zip', '')
        string = ' '.join(re.findall('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', string))  # splits camelCase into camel case
        string = ' '.join(re.split('(\s+|^)([A-Za-z]+)\d+(\s+|$)', string))  # removes number at the end
        string = string.lower()
        string = string.replace('_', ' ')
        string = string.replace('-', ' ')
        string = string.replace('[', ' ')
        string = string.replace(']', ' ')
        string = string.replace('(', ' ')
        string = string.replace(')', ' ')
        string = string.replace('?', ' ')
        string = string.replace('*', ' ')
        string = string.replace('copy of', ' ')
        string = string.replace('copy', ' ')
        string = " ".join(string.split())

        part_names.append(string)

    random.shuffle(part_names)
    return list(set(part_names))


data['part_names_clean'] = data.apply(lambda row: process_part_names(row.part_names), axis=1)
data.dropna(subset=['part_names_clean'], inplace=True)

In [62]:
data.head()

,assembly_name,part_names,assembly_name_clean,part_names_clean
0,Lava Lamp 2,"[Blob3, Blob2, Blob1, Glass, Cap]",lava lamp,"[cap, blob, glass]"
1,Sample - Headphones,"[Pivot hinge, Headphone hinge, Telescope hinge...",headphones,"[upper band, pivot hinge, telescope hinge, hea..."
4,Coffee Mug,"[Mug, Lid]",coffee mug,"[mug, lid]"
5,Dave's Handsome Mug,"[Lid, Seal, Vessel]",dave's handsome mug,"[vessel, seal, lid]"
9,Mechanical Pencil,"[Eraser, Pencil Lead, Rubber Grip, Gripper Rod...",mechanical pencil,"[button release, eraser, pencil lead, gripper ..."


In [63]:
data.head()

,assembly_name,part_names,assembly_name_clean,part_names_clean
0,Lava Lamp 2,"[Blob3, Blob2, Blob1, Glass, Cap]",lava lamp,"[cap, blob, glass]"
1,Sample - Headphones,"[Pivot hinge, Headphone hinge, Telescope hinge...",headphones,"[upper band, pivot hinge, telescope hinge, hea..."
4,Coffee Mug,"[Mug, Lid]",coffee mug,"[mug, lid]"
5,Dave's Handsome Mug,"[Lid, Seal, Vessel]",dave's handsome mug,"[vessel, seal, lid]"
9,Mechanical Pencil,"[Eraser, Pencil Lead, Rubber Grip, Gripper Rod...",mechanical pencil,"[button release, eraser, pencil lead, gripper ..."


## Create sentences

In [64]:
sentence_pairs = []
for index, row in tqdm(data.iterrows(), total=len(data)):
    num_parts = len(row['part_names_clean'])
    if num_parts > 1:
        sentence_1 = f"An assembly named '{row['assembly_name_clean']}' containing the following parts: "
        for part_name in row['part_names_clean'][:num_parts-1]:
                sentence_1 += f"{part_name}, "


        sentence_2 = ''
        for part_name in row['part_names_clean'][num_parts-1:]:
            sentence_2 += f"{part_name}, "
        sentence_2 = sentence_2[:-2] + "."

        sentence_pairs.append([sentence_1, sentence_2])


100%|██████████| 61601/61601 [00:03<00:00, 17062.34it/s]


In [65]:
random.shuffle(sentence_pairs)
print(len(sentence_pairs))
#pprint(sentence_pairs[:50])

40240


# Train test split

In [66]:
positive_samples = sentence_pairs[:len(sentence_pairs)//2]
negative_samples = sentence_pairs[len(sentence_pairs)//2:]
#positive_samples = sentence_pairs[:len(sentence_pairs)]
#negative_samples = sentence_pairs[:len(sentence_pairs)]

In [67]:
# Scramble the negative samples while maintaining the sentence pair order
print(negative_samples[1])
print(positive_samples[1])

negative_sentence_1 = [pair[0] for pair in negative_samples]
negative_sentence_2 = [pair[1] for pair in negative_samples]
random.shuffle(negative_sentence_2)
negative_samples = [list(x) for x in zip(negative_sentence_1, negative_sentence_2)]
print(negative_samples[1])

["An assembly named 'c coupler' containing the following parts: coupler, nut, ", 'nut 14/17.']
["An assembly named 'drone king of the hill' containing the following parts: light sensor, plug, tube, platform, ", 'casing.']
["An assembly named 'c coupler' containing the following parts: coupler, nut, ", 'female connector.']


In [68]:
# Create labels: 1 = positive, 0 = negative
positive_samples = [(sample, 1) for sample in positive_samples]
negative_samples = [(sample, 0) for sample in negative_samples]

sentence_pairs = positive_samples + negative_samples

In [69]:
#negative_samples

In [70]:
train, test = train_test_split(sentence_pairs, test_size=0.2)
print(f"Length of train: {len(train)}")
print(f"Length of test: {len(test)}")

Length of train: 32192
Length of test: 8048


In [71]:
#train[:5]

In [72]:
# Save out train data

train_data_path = "/content/drive/My Drive/W266/entailment/train_entailment.csv"
train_df = pd.DataFrame(train)

if not os.path.exists(Path(train_data_path).parent):
    os.mkdir(Path(train_data_path).parent)

train_df.to_csv(train_data_path, index=False)

In [73]:
def preprocess_data(data, tokenizer, max_length=128):
    labels = [item[1] for item in data]
    sentence_pairs = [item[0] for item in data]

    # With BERT tokenizer's batch_encode_plus, sentence pairs are
    # encoded together and separated by [SEP] token.
    encoded = tokenizer.batch_encode_plus(
        sentence_pairs,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_token_type_ids=True,
        return_tensors="tf"
    )

    # Extract encoded features and labels, add to corresponding lists
    input_ids = np.array(encoded["input_ids"], dtype="int32")
    attention_masks = np.array(encoded["attention_mask"], dtype="int32")
    token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")
    
    return [input_ids, attention_masks, token_type_ids], np.array(labels)

# Construct the model

In [74]:
bert_model_name='bert-base-uncased'
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
#bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#bert_model = TFBertModel.from_pretrained('bert-base-uncased')
bert_model = TFAutoModel.from_pretrained(bert_model_name)
bert_model.trainable = False

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [90]:
def build_baseline_model(bert_model, max_length=128):
    input_ids = layers.Input(shape=(max_length), dtype=tf.int32, name='input_ids')
    attention_masks = layers.Input(shape=(max_length), dtype=tf.int32, name='attention_masks')
    token_type_ids = layers.Input(shape=(max_length), dtype=tf.int32, name='token_type_ids')

    bert_output = bert_model(input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids)
    sequence_output=bert_output[1]
    #sequence_output = bert_output.last_hidden_state

    #attn_output = layers.MultiHeadAttention(num_heads=8, key_dim=16)(sequence_output, sequence_output)
    #max_pool = layers.GlobalMaxPooling1D()(sequence_output, sequence_output)
    dropout_output = layers.Dropout(0)(sequence_output)
    final_output = layers.Dense(1, activation="sigmoid")(dropout_output)

    model = tf.keras.models.Model(inputs=[input_ids, attention_masks, token_type_ids],
                                  outputs=[final_output])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [24]:
def build_New_model(bert_model, train_layers=-1, max_length=128):

    if train_layers == -1:
        # Freeze all layers of pre-trained BERT model
        bert_model.trainable = False

    else:
        # Restrict training to the train_layers outer transformer layers
        retrain_layers = []

        for retrain_layer_number in range(train_layers):

            layer_code = '_' + str(11 - retrain_layer_number)
            retrain_layers.append(layer_code)
          
        
        print('retrain layers: ', retrain_layers)

        for w in bert_model.weights:
            if not any([x in w.name for x in retrain_layers]):
                print('freezing: ', w)
                w._trainable = False

    input_ids = layers.Input(shape=(max_length), dtype=tf.int32, name='input_ids')
    attention_masks = layers.Input(shape=(max_length), dtype=tf.int32, name='attention_masks')
    token_type_ids = layers.Input(shape=(max_length), dtype=tf.int32, name='token_type_ids')

    bert_output = bert_model(input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids)
    sequence_output=bert_output[1]
    #sequence_output = bert_output.last_hidden_state

    #attn_output = layers.MultiHeadAttention(num_heads=4, key_dim=100)(sequence_output, sequence_output)
    #max_pool = layers.GlobalMaxPooling1D()(attn_output)
    #dropout_output = layers.Dropout(0.3)(max_pool)
    final_output = layers.Dense(1, activation="sigmoid")(sequence_output)

    model = tf.keras.models.Model(inputs=[input_ids, attention_masks, token_type_ids],
                                  outputs=[final_output])
    model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
def build_New_model2(bert_model, max_length=128, hidden_dim=256):
    input_ids = layers.Input(shape=(max_length), dtype=tf.int32, name='input_ids')
    attention_masks = layers.Input(shape=(max_length), dtype=tf.int32, name='attention_masks')
    token_type_ids = layers.Input(shape=(max_length), dtype=tf.int32, name='token_type_ids')

    bert_output = bert_model(input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids)
    sequence_output = bert_output.last_hidden_state

    hidden = tf.keras.layers.Dense(hidden_dim, activation='relu', name='hidden_layer')(sequence_output)

    hidden = tf.keras.layers.Dense(hidden_dim/2, activation='relu', name='hidden_layer2')(hidden)

    attn_output = layers.MultiHeadAttention(num_heads=8, key_dim=100)(hidden, hidden)
    max_pool = layers.GlobalMaxPooling1D()(attn_output)
    dropout_output = layers.Dropout(0.3)(max_pool)
    final_output = layers.Dense(1, activation="sigmoid")(dropout_output)

    model = tf.keras.models.Model(inputs=[input_ids, attention_masks, token_type_ids],
                                  outputs=[final_output])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [93]:
def build_New_model3(bert_model, max_length=128, hidden_dim=256):
    input_ids = layers.Input(shape=(max_length), dtype=tf.int32, name='input_ids')
    attention_masks = layers.Input(shape=(max_length), dtype=tf.int32, name='attention_masks')
    token_type_ids = layers.Input(shape=(max_length), dtype=tf.int32, name='token_type_ids')

    bert_output = bert_model(input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids)
    sequence_output = bert_output.last_hidden_state
    print(sequence_output)
    attn_output = layers.MultiHeadAttention(num_heads=8, key_dim=32)(sequence_output, sequence_output)

    #hidden = tf.keras.layers.Dense(hidden_dim, activation='relu', name='hidden_layer')(attn_output)

    #hidden = tf.keras.layers.Dense(hidden_dim, activation='relu', name='hidden_layer2')(hidden)

    max_pool = layers.GlobalMaxPooling1D()(attn_output)
    dropout_output = layers.Dropout(0)(max_pool)
    final_output = layers.Dense(1, activation="sigmoid")(dropout_output)

    model = tf.keras.models.Model(inputs=[input_ids, attention_masks, token_type_ids],
                                  outputs=[final_output])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [91]:
model = build_baseline_model(bert_model, max_length=256)
#model1 = build_baseline_model(bert_model, max_length=128)


In [ ]:
model1=build_baseline_model(bert_model, max_length=256)

In [94]:
model2= build_New_model3(bert_model, max_length=256, hidden_dim=256)

KerasTensor(type_spec=TensorSpec(shape=(None, 256, 768), dtype=tf.float32, name=None), name='tf_bert_model_2/bert/encoder/layer_._11/output/LayerNorm/batchnorm/add_1:0', description="created by layer 'tf_bert_model_2'")


In [ ]:
#keras.utils.plot_model(model1, show_shapes=True, dpi=90)

In [77]:
class SNLIDataGeneratorFromFile(tf.keras.utils.Sequence):
    def __init__(self,
                 tokenizer,
                 n_examples,
                 data_filename,
                 max_length=128,
                 batch_size=32,
                 shuffle=True):
        
        self.tokenizer = tokenizer
        self.n_examples = n_examples
        self.data_filename = data_filename
        self.max_length = max_length
        self.batch_size = batch_size
        self.shuffle = shuffle
        
        # Initialize row order, call on_epoch_end to shuffle row indices
        self.row_order = np.arange(1, self.n_examples+1)
        self.on_epoch_end()
    
    def __len__(self):
        return self.n_examples // self.batch_size

    def __getitem__(self, idx):
        batch_start = idx * self.batch_size
        batch_end = (idx + 1) * self.batch_size

        # Indices to skip are the ones in the shuffled row_order before and
        # after the chunk we'll use for this batch
        batch_idx_skip = self.row_order[:batch_start] + self.row_order[batch_end:]
        train_data_df = pd.read_csv(self.data_filename, skiprows=batch_idx_skip)

        train_data = []
        for index, row in train_data_df.iterrows():
            train_data.append((row[0], row[1]))

        batch_data = preprocess_data(
            train_data,
            self.tokenizer,
            self.max_length
        )

        return batch_data
    
    def on_epoch_end(self):
        if self.shuffle:
            self.row_order = list(np.random.permutation(self.row_order))

In [78]:
train_data_generator = SNLIDataGeneratorFromFile(
    tokenizer=bert_tokenizer,
    n_examples=len(train),
    data_filename=train_data_path,
    batch_size=16,
    max_length=256
)

In [79]:
test_data = preprocess_data(
    test, tokenizer=bert_tokenizer, max_length=256
)

In [80]:
checkpoint_dir = '/content/drive/My Drive/W266/entailment/'
checkpoint_filepath = checkpoint_dir + 'weights.{epoch:02d}-{val_accuracy:.2f}.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True)

## Train model

In [81]:
tf.config.experimental_run_functions_eagerly(True)

In [31]:
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)


In [92]:
history=model.fit(train_data_generator, validation_data=test_data, epochs=2,
          callbacks=[model_checkpoint_callback])

2012/2012 [==============================] - 876s 435ms/step - loss: 0.6911 - accuracy: 0.5460 - val_loss: 0.5977 - val_accuracy: 0.6213


In [ ]:
#history=model1.fit(train_data_generator, validation_data=test_data, epochs=2,
#          callbacks=[model_checkpoint_callback])

In [ ]:
#history=model2.fit(train_data_generator, validation_data=test_data, epochs=2,
#          callbacks=[model_checkpoint_callback])

In [ ]:
#history=model2.fit(train_data_generator, validation_data=test_data, epochs=2,
#          callbacks=[model_checkpoint_callback])

In [95]:
history=model2.fit(train_data_generator, validation_data=test_data, epochs=2,
          callbacks=[model_checkpoint_callback])

2012/2012 [==============================] - 885s 440ms/step - loss: 0.5300 - accuracy: 0.7213 - val_loss: 0.6039 - val_accuracy: 0.6787


In [ ]:
def make_plot(axs, history1, 
              history2, 
              y_lim_loss_lower=0.4, 
              y_lim_loss_upper=0.6,
              y_lim_accuracy_lower=0.7, 
              y_lim_accuracy_upper=0.8,
              model_1_name='model 1',
              model_2_name='model 2',
              
             ):
    box = dict(facecolor='yellow', pad=5, alpha=0.2)

    ax1 = axs[0, 0]
    ax1.plot(history1.history['loss'])
    ax1.plot(history1.history['val_loss'])
    ax1.set_title('loss - ' + model_1_name)
    ax1.set_ylabel('loss', bbox=box)
    ax1.set_ylim(y_lim_loss_lower, y_lim_loss_upper)

    ax3 = axs[1, 0]
    ax3.set_title('accuracy - ' + model_1_name)
    ax3.plot(history1.history['accuracy'])
    ax3.plot(history1.history['val_accuracy'])
    ax3.set_ylabel('accuracy', bbox=box)
    ax3.set_ylim(y_lim_accuracy_lower, y_lim_accuracy_upper)


    ax2 = axs[0, 1]
    ax2.set_title('loss - ' + model_2_name)
    ax2.plot(history2.history['loss'])
    ax2.plot(history2.history['val_loss'])
    ax2.set_ylim(y_lim_loss_lower, y_lim_loss_upper)

    ax4 = axs[1, 1]
    ax4.set_title('accuracy - ' + model_2_name)

    # small adjustment to account for the 2 accuracy measures in the Weighted Averging Model with Attention
    if 'classification_accuracy' in history2.history.keys():
      ax4.plot(history2.history['classification_accuracy'])
    else:
      ax4.plot(history2.history['accuracy'])
    
    if 'val_classification_accuracy' in history2.history.keys():
      ax4.plot(history2.history['val_classification_accuracy'])
    else:
      ax4.plot(history2.history['val_accuracy'])
    ax4.set_ylim(y_lim_accuracy_lower, y_lim_accuracy_upper)

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(2, 2)
fig.subplots_adjust(left=0.2, wspace=0.6)
make_plot(axs, history, history,model_1_name='Plots', model_2_name='Plots')

fig.align_ylabels(axs[:, 1])
fig.set_size_inches(18.5, 10.5)
plt.show()